***

**<center><font size = "5">Pandas GroupBy<center>**
**<center><font size = "4">Introduction and Real World Examples<center>**
***
<center><font size = "2">Prepared by: Kurt Klingensmith<center>

***

# Notebook Setup

## Libraries:

In [51]:
# Data Handling
import pandas as pd
import numpy as np

# Data visualization
import plotly.express as px

# Introduction to GroupBy

## Load in Dataset and Execute Basic "Hello World" Function:

In [52]:
# Load Data:
df = pd.read_csv('StudentData.csv')
df.head(3)

,Student,Age,Test1,Test2,Test3,ClassSession,Grade,LetterGrade,CoursePass
0,Ashley Sanchez,22,87,76,75,Morning,79.3,C,Yes
1,Joshua Garcia,24,76,81,70,Afternoon,75.7,C,Yes
2,Sara Mendoza,21,61,69,85,Evening,71.7,C,Yes


In [53]:
# Use groupby to see pass breakdown:
df.groupby('CoursePass')['CoursePass'].count()

CoursePass
No     10
Yes    90
Name: CoursePass, dtype: int64

In [54]:
# Compare to "value counts" method:
df.CoursePass.value_counts()

Yes    90
No     10
Name: CoursePass, dtype: int64

In [55]:
df.shape

(100, 9)

## GroupBy Multiple Columns:

In [56]:
# Show example of multiple columns grouped by two columns:
df.groupby(['ClassSession', 'CoursePass'])['CoursePass'].count()

ClassSession  CoursePass
Afternoon     No             1
              Yes           39
Evening       No             9
              Yes           24
Morning       Yes           27
Name: CoursePass, dtype: int64

In [58]:
# Changing column order changes output:
df.groupby(['CoursePass', 'ClassSession'])['ClassSession'].count()

CoursePass  ClassSession
No          Afternoon        1
            Evening          9
Yes         Afternoon       39
            Evening         24
            Morning         27
Name: ClassSession, dtype: int64

## Make the Output Look Nicer

In [59]:
# Make the output into a dataframe:
df.groupby(['ClassSession', 'CoursePass'])['CoursePass'].count().to_frame()

CoursePass
ClassSession CoursePass            
Afternoon    No                   1
             Yes                 39
Evening      No                   9
             Yes                 24
Morning      Yes                 27

In [60]:
# Use reset_index() to achieve single-level column names:
df.groupby(['ClassSession', 'CoursePass'])['CoursePass'].count().reset_index(name='Count')

,ClassSession,CoursePass,Count
0,Afternoon,No,1
1,Afternoon,Yes,39
2,Evening,No,9
3,Evening,Yes,24
4,Morning,Yes,27


In [61]:
# Hide index:
df.groupby(['ClassSession', 'CoursePass'])['CoursePass'].count().reset_index(name='Count').style.hide_index()

ClassSession,CoursePass,Count
Afternoon,No,1
Afternoon,Yes,39
Evening,No,9
Evening,Yes,24
Morning,Yes,27


## Additional Capabilities

In [62]:
# Create a groupby object:
dfGroupby = df.groupby(['LetterGrade', 'CoursePass'])
dfGroupby[['Test1', 'Test2']].mean()

,,Test1,Test2
LetterGrade,CoursePass,,
A,Yes,90.857143,90.428571
B,Yes,82.970588,84.294118
C,Yes,74.959184,73.530612
D,No,61.222222,66.666667
F,No,56.000000,62.000000


In [63]:
# Find the average age and test scores by Letter Grade achieved:
dfGroupby = df.groupby(['LetterGrade'])
dfGroupby[['Age', 'Test1', 'Test2', 'Test3']].mean()

,Age,Test1,Test2,Test3
LetterGrade,,,,
A,22.571429,90.857143,90.428571,92.714286
B,22.000000,82.970588,84.294118,85.029412
C,21.775510,74.959184,73.530612,75.959184
D,21.333333,61.222222,66.666667,69.555556
F,20.000000,56.000000,62.000000,61.000000


In [64]:
# Use the groupby object with multiple aggregations:
dfGroupby = df.groupby(['LetterGrade', 'CoursePass'])
dfGroupby[['Grade', 'Age']].agg(['mean', 'min', 'max']).reset_index()

LetterGrade CoursePass      Grade                    Age        
                               mean   min   max       mean min max
0           A        Yes  91.328571  90.3  92.7  22.571429  20  24
1           B        Yes  84.094118  80.0  89.7  22.000000  20  24
2           C        Yes  74.816327  70.0  79.3  21.775510  20  24
3           D         No  65.822222  60.7  69.3  21.333333  20  24
4           F         No  59.700000  59.7  59.7  20.000000  20  20

In [65]:
# Show use of median() and multiple aggregations:
df.groupby('CoursePass', as_index=False)[['Age', 'Grade']].median().style.hide_index()

CoursePass,Age,Grade
No,21,66.000000
Yes,22,78.500000


In [66]:
# Show an example of min():
df.groupby(['CoursePass', 'ClassSession'])['Grade'].min().reset_index().style.hide_index()

CoursePass,ClassSession,Grade
No,Afternoon,69.000000
No,Evening,59.700000
Yes,Afternoon,70.700000
Yes,Evening,70.000000
Yes,Morning,72.700000


In [67]:
# Show use of agg() function:
df.groupby(['CoursePass', 'ClassSession']).agg({'Age':'mean', 'Grade':'mean'})

Age      Grade
CoursePass ClassSession                      
No         Afternoon     23.000000  69.000000
           Evening       21.000000  64.788889
Yes        Afternoon     22.000000  80.861538
           Evening       21.333333  73.554167
           Morning       22.333333  83.170370

In [68]:
# Show use of .agg with numpy:
df.groupby(['CoursePass', 'ClassSession'])['Grade'].agg([np.mean, np.max, np.min])

mean  amax  amin
CoursePass ClassSession                       
No         Afternoon     69.000000  69.0  69.0
           Evening       64.788889  69.3  59.7
Yes        Afternoon     80.861538  92.7  70.7
           Evening       73.554167  83.0  70.0
           Morning       83.170370  92.3  72.7

In [69]:
# Show another example with .agg and column renaming:
data = df.groupby(['CoursePass', 'ClassSession'])['Grade'].agg([np.mean, np.max, np.min]).reset_index()

data = data.rename(columns={'mean':'AverageGrade',
                            'amax':'HighestGrade',
                            'amin':'LowestGrade'})

data.style.hide_index()

CoursePass,ClassSession,AverageGrade,HighestGrade,LowestGrade
No,Afternoon,69.000000,69.000000,69.000000
No,Evening,64.788889,69.300000,59.700000
Yes,Afternoon,80.861538,92.700000,70.700000
Yes,Evening,73.554167,83.000000,70.000000
Yes,Morning,83.170370,92.300000,72.700000


In [70]:
# Use the agg function to directly name columns:
df.groupby(['CoursePass', 'ClassSession'])\
        .agg(AverageGrade=('Grade', 'mean'),
             HighestGrade=('Grade', 'max'),
             LowestGrade=('Grade', 'min'))

AverageGrade  HighestGrade  LowestGrade
CoursePass ClassSession                                         
No         Afternoon        69.000000          69.0         69.0
           Evening          64.788889          69.3         59.7
Yes        Afternoon        80.861538          92.7         70.7
           Evening          73.554167          83.0         70.0
           Morning          83.170370          92.3         72.7

# Example 2: Use GroupBy to Quickly Visualize Grade Counts

In [71]:
df.head()

,Student,Age,Test1,Test2,Test3,ClassSession,Grade,LetterGrade,CoursePass
0,Ashley Sanchez,22,87,76,75,Morning,79.3,C,Yes
1,Joshua Garcia,24,76,81,70,Afternoon,75.7,C,Yes
2,Sara Mendoza,21,61,69,85,Evening,71.7,C,Yes
3,Michael Morris,24,87,75,86,Morning,82.7,B,Yes
4,Jessica Ward,23,60,79,78,Evening,72.3,C,Yes


In [72]:
session = df.groupby(['ClassSession', 'LetterGrade'])['LetterGrade'].count().reset_index(name='Count')
session = session.sort_values(by='LetterGrade', ascending = True)
session

,ClassSession,LetterGrade,Count
0,Afternoon,A,2
8,Morning,A,5
1,Afternoon,B,20
4,Evening,B,1
9,Morning,B,13
2,Afternoon,C,17
5,Evening,C,23
10,Morning,C,9
3,Afternoon,D,1
6,Evening,D,8


In [73]:
# Generate plot:
plot = px.histogram(session,
               y='Count',
               x='LetterGrade',
               color='ClassSession')
plot.update_layout(
    title={'text': "Grade Distribution\
                    <br><sup>Number of Grades by Type and Class Session</br>",
           'xanchor': 'center',
           'yanchor': 'top',
           'x': 0.47},
    xaxis_title='Letter Grade',
    yaxis_title='Count')
plot.show()

***